In [72]:
# assume @ is the epsilon symbol
class edge:
    def __init__(self,src,dist,label="@"):
        self.src = src
        self.dist = dist
        self.label = label

In [73]:
sid=0

class state:
    def __init__(self,edges):
        global sid
        self.Outedges:list[edge] = edges
        self.label = "S"+str(sid)
        sid+=1
    def add_edge(self,edge):
        self.Outedges.append(edge)

In [74]:
from graphviz import Digraph

class NFA:
    def __init__(self,start,accept,inner_states):
        self.start:state = start
        self.accept:state = accept
        self.inner_states:list[state] =inner_states

    def display(self):
        with open('nfa.json', 'w') as f:
            f.write("{\n")
            f.write("\"startingState\":\""+self.start.label+"\",\n")
            for s in self.inner_states:
                f.write("\""+s.label+"\":\n")
                f.write("   {\n")
                f.write("       \"isTerminatingState\":\""+str(s.label==self.accept.label)+"\"")
                if len(s.Outedges)>0:
                    f.write(",\n")
                for e in s.Outedges:
                    f.write("       \""+e.label+"\":\""+e.dist.label+"\"")
                    if e!=s.Outedges[-1]:
                        f.write(",\n")
                    else:
                        f.write("\n")
                f.write("   }")
                if s!=self.inner_states[-1]:
                    f.write(",\n")
                else:
                    f.write("\n")
            f.write("}\n")

    def graph (self):
        dot = Digraph(comment='NFA graph',format='png',graph_attr={ 'rankdir': 'LR'})
        # dot.node(self.start.label,self.start.label,shape="circle")
        # dot.node(self.accept.label,self.accept.label,shape="doublecircle")
        for s in self.inner_states:
            if s.label==self.accept.label:
                dot.node(s.label,s.label,shape="doublecircle")
            else:
                dot.node(s.label,s.label,shape="circle")
            for e in s.Outedges:
                dot.edge(e.src.label,e.dist.label,e.label)
        dot.render('test-output/round-table.gv', view=True)
        


In [75]:
def Concat(stack:list[NFA]):
    nfa2:NFA = stack.pop()
    nfa1:NFA = stack.pop()
    
    newEdges = edge(nfa1.accept,nfa2.start)
    nfa1.accept.add_edge(newEdges)

    resNfa = NFA(nfa1.start,nfa2.accept,nfa1.inner_states+nfa2.inner_states)
    stack.append(resNfa)

    return stack

In [76]:
def Or(stack:list[NFA]):
    nfa1 = stack.pop()
    nfa2 = stack.pop()
    
    newStart = state([])
    # sid+=1
    newEnd = state([])
    # sid+=1
    
    newEdges1 = edge(newStart,nfa1.start)
    newEdges2 = edge(newStart,nfa2.start)

    newStart.add_edge(newEdges1)
    newStart.add_edge(newEdges2)

    newEdges3 = edge(nfa1.accept,newEnd)
    newEdges4 = edge(nfa2.accept,newEnd)
    
    nfa1.accept.add_edge(newEdges3)
    nfa2.accept.add_edge(newEdges4)

    resNfa = NFA(newStart,newEnd,nfa1.inner_states+nfa2.inner_states+[newStart,newEnd])
    stack.append(resNfa)
    
    return stack

In [77]:
def ZeroMore(stack:list[NFA]):
    nfa = stack.pop()
    
    newEdges1 = edge(nfa.accept,nfa.start)
    nfa.accept.add_edge(newEdges1)

    newStart = state([])
    # sid+=1
    newEnd = state([])
    # sid+=1

    newEdges2 = edge(newStart,nfa.start)
    newStart.add_edge(newEdges2)

    newEdges3 = edge(nfa.accept,newEnd)
    nfa.accept.add_edge(newEdges3)

    newEdges4 = edge(newStart,newEnd)
    newStart.add_edge(newEdges4)

    resNfa = NFA(newStart,newEnd,nfa.inner_states+[newStart,newEnd])
    stack.append(resNfa)
    
    return stack

In [78]:
def OneMore(stack:list[NFA]):
    nfa = stack.pop()
    
    newEdges1 = edge(nfa.accept,nfa.start)
    nfa.accept.add_edge(newEdges1)

    newStart = state([])
    # sid+=1
    newEnd = state([])
    # sid+=1

    newEdges2 = edge(newStart,nfa.start)
    newStart.add_edge(newEdges2)

    newEdges3 = edge(nfa.accept,newEnd)
    nfa.accept.add_edge(newEdges3)

    resNfa = NFA(newStart,newEnd,nfa.inner_states+[newStart,newEnd])
    stack.append(resNfa)
    
    return stack

In [79]:
def ZeroOne(stack:list[NFA]):
    nfa = stack.pop()

    newStart = state([])
    # sid+=1
    newEnd = state([])
    # sid+=1

    newEdges1 = edge(newStart,nfa.start)
    newStart.add_edge(newEdges1)

    newEdges2 = edge(nfa.accept,newEnd)
    nfa.accept.add_edge(newEdges2)

    newEdges3 = edge(newStart,newEnd)
    newStart.add_edge(newEdges3)

    resNfa = NFA(newStart,newEnd,nfa.inner_states+[newStart,newEnd])
    stack.append(resNfa)
    
    return stack

In [80]:
def Construct(stack:list[NFA],s:str):
    
    start=state([])
    # sid+=1
    accept=state([])
    # sid+=1
    
    newEdges = edge(start,accept,s)
    start.add_edge(newEdges)

    resNfa = NFA(start,accept,[start,accept])
    stack.append(resNfa)

    return stack

In [81]:
def PostfixToNFA(postfix:str):
    alphanum = {char:char for char in "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789."}
    stack = []
    for c in postfix:
        if c in alphanum:
            stack = Construct(stack,c)
        elif c == "*":
            stack = ZeroMore(stack)
        elif c == "+":
            stack = OneMore(stack)
        elif c == "?":
            stack = ZeroOne(stack)
        elif c == "&":
            stack = Concat(stack)
        elif c == "|":
            stack = Or(stack)
        else:
            print("Invalid postfix expression character {c} is not recognized")
            return None
    if len(stack)!=1:
        print("Invalid postfix expression")
        return None
    
    result = stack.pop()
        
    return result

In [82]:
# test cell 
# expression = "(A+B*)?(C|D)"
# postfix = "A+B*.?CD|."
postfix="A+B*&?CD|&"
nfa = PostfixToNFA(postfix)
nfa.display()
# nfa.graph()


startingState: S8 ,
S0 : {
 isTerminatingState :  False ,    
A : S1 ,
},

S1 : {
 isTerminatingState :  False ,    
@ : S0 ,
@ : S3 ,
},

S2 : {
 isTerminatingState :  False ,    
@ : S0 ,
},

S3 : {
 isTerminatingState :  False ,    
@ : S6 ,
},

S4 : {
 isTerminatingState :  False ,    
B : S5 ,
},

S5 : {
 isTerminatingState :  False ,    
@ : S4 ,
@ : S7 ,
},

S6 : {
 isTerminatingState :  False ,    
@ : S4 ,
@ : S7 ,
},

S7 : {
 isTerminatingState :  False ,    
@ : S9 ,
},

S8 : {
 isTerminatingState :  False ,    
@ : S2 ,
@ : S9 ,
},

S9 : {
 isTerminatingState :  False ,    
@ : S14 ,
},

S12 : {
 isTerminatingState :  False ,    
D : S13 ,
},

S13 : {
 isTerminatingState :  False ,    
@ : S15 ,
},

S10 : {
 isTerminatingState :  False ,    
C : S11 ,
},

S11 : {
 isTerminatingState :  False ,    
@ : S15 ,
},

S14 : {
 isTerminatingState :  False ,    
@ : S12 ,
@ : S10 ,
},

S15 : {
 isTerminatingState :  True ,    
},

